In [ ]:
import pandas as pd
import numpy as np
from sklearn import svm
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import classification_report
from scipy import stats
import time

# Load the dataset from a CSV file
data = pd.read_csv("result.csv", sep=",")

# Define the features (explanatory variables) for training
columns =['raw(m)','raw(s)']
data_x = data.loc[:, columns].values

# Define the target variable
data_y = data['result'].values

# Split the data into training and testing sets (70% training, 30% testing)
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.3, random_state=22222)

# Set up the parameter distribution for RandomizedSearchCV
params = {"C": stats.expon(scale=1), "gamma": stats.expon(scale=0.001)}

# Perform hyperparameter tuning using RandomizedSearchCV
clf = RandomizedSearchCV(svm.SVC(), params, cv=5, return_train_score=True, n_iter=100, scoring="accuracy")
clf.fit(train_x, train_y)

# Retrieve the best model found by RandomizedSearchCV
best_clf = clf.best_estimator_
print('\nBest Classifier:')
print(best_clf)
print('')

# Perform cross-validation and print the scores
scores = cross_val_score(best_clf, train_x, train_y, cv=5)
print("Cross-Validation Scores:", scores)
print("Mean Cross-Validation Score:", np.mean(scores))

# Save the best model to a file
joblib.dump(best_clf, 'train.learn')

# Load the saved model for evaluation
clf2 = joblib.load('train.learn')

# Predict the training and testing sets
predtrain_y = clf2.predict(train_x)
predtest_y = clf2.predict(test_x)

# Print classification reports for both training and testing sets
print('\nTraining Set Classification Report:')
print(classification_report(train_y, predtrain_y, target_names=["Retry", "Pass"]))
print('Testing Set Classification Report:')
print(classification_report(test_y, predtest_y, target_names=["Retry", "Pass"]))

# Display the true and predicted values for the testing set
print('Test Labels:')
print(test_y)     
print('Predicted Labels:')
print(predtest_y)